In [1]:
# Imports
import os

In [79]:
# Compiler Statements

LAST_CELL = 64

VARS = {
}

COMMANDS = {
    "TEST": "Test",
    ":=": "Assign",
    #"+": "Add",
    #"print": "Print",
    #"=": "Equals",
}

In [80]:
# IO functions

def read_input_file(filepath: str):
    with open(filepath) as f:
        lines = f.readlines()
    f.close()
    return lines

def write_output_file(program: str, filepath: str):
    f = open(filepath, "w")
    f.write(program)
    f.close()
    return


# Compiler

def compile(lines: str, name: str, path: str):
    program = f";==={name}===\n"
    for line in lines:
        program += compile_line(line)
    program += 'end'
    write_output_file(program, path)
    pass

def compile_line(line: str):
    res = find_and_execute_command(line)
    return res + "\n"

def tokenize_line(line: str):
    tokens = line.split()
    return tokens

def get_last_cell():
    global LAST_CELL
    return f"${LAST_CELL}"

def bump_last_cell(amount: int = 1):
    global LAST_CELL
    LAST_CELL += amount

    if(LAST_CELL > 71):
        raise StopIteration("Ram exceeded")

    return f"${LAST_CELL}"

def create_var(var: str):
    global VARS
    VARS.update({var: bump_last_cell()})
    pass

def get_var(var: str):
    global VARS

    print("Var Added")

    if(var not in VARS):
        create_var(var)

    return VARS[var]


# Method execution

def get_method(name: str):
    possibles = globals().copy()
    possibles.update(locals())
    method = possibles.get(name)
    if not method:
        raise NotImplementedError("Method %s not implemented" % name)
    return method


def find_and_execute_command(line: str):
    tokens = tokenize_line(line)
    name = ""
    for cmd in COMMANDS:
        if cmd in tokens:
            name = COMMANDS[cmd]

    if not name:
        print("No command in line")
        return ""

    method = get_method(name)
    res = method(line)
    if not res:
        res = ""
    return res

# Commands

def Assign(line: str):
    tokens = tokenize_line(line)
    asm = ""
    
    asm += get_var(tokens[0])
    return asm

def Test(line: str):
    return "Test"

In [82]:
# TESTS

# Read File
lines = read_input_file("../tests/test.pass")

# Compile
compile(lines, "TEST", "../tests/test.ass")

print(VARS)

Var Added
Var Added
{'QA': '64', 'AYAYA': '$66', 'AQ': '$67'}
